In [11]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import random
import numpy.random as rng
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Input, Lambda
from keras.layers import Flatten, Dense, BatchNormalization, Activation, Dropout
from keras.layers.advanced_activations import PReLU
from keras.models import Sequential, load_model, Model
from keras.regularizers import l2
from keras.losses import binary_crossentropy

import os
from os.path import join, dirname, isfile, abspath
from keras.preprocessing.image import img_to_array, load_img #convert image to array

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(1337)  # for the sake of reproducibility

def W_INIT(shape,name=None):
    """Initialize weights as in paper"""
    values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)

#CNN_MODEL_FILE = join(MODELS_DIR, 'cnn.h5')
nb_epoch = 10
BATCH_SIZE = 48
L2_REG = 0.03
#W_INIT = 'he_normal'
LAST_FEATURE_MAPS_LAYER = 46
LAST_FEATURE_MAPS_SIZE = (128, 8, 8)
PENULTIMATE_LAYER = 51
PENULTIMATE_SIZE = 2048
SOFTMAX_LAYER = 55
SOFTMAX_SIZE = 475

def readin_small_data():
    """Read small data set into a 2D numpy array
    return: numpy array contains the small data set, shape: (50,000, 230)
    """
    PATH = os.getcwd()
    DATA = os.path.join(PATH, 'orange_small_train','orange_small_train.data')
    with open(DATA) as f:
        header = f.readline().strip('\n').split('\t')
    
    data_type = {key:np.float64 for key in header[:190]}
    data_type.update({key:str for key in header[190:]})
    return pd.read_table(DATA, dtype=data_type)

def readin_label(target):
    """Read upselling, churn or appetency label into a 2D numpy array
    parameter target: choose which label to load
    return: numpy array contains the specified label of small data set, shape: (50,000, 1)
    """
    assert target in ['upselling', 'churn', 'appetency']
    PATH = os.getcwd()
    DATA = os.path.join(PATH, 'orange_small_train','orange_small_train_'+target+'.labels.txt')

    with open(DATA) as f:
        lines = f.readlines()
        data = list(map(lambda x: True if x.strip() == '1' else False, lines))

    return np.array(data)[:,np.newaxis]

model.add(BatchNormalization(axis=1, mode=0))
    model.add(PReLU(init=W_INIT))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(p=0.5))
    model.add(Flatten())

In [19]:
def dnn(data_dim):
    model = Sequential()
    
    model.add(Dense(32, input_shape=(data_dim)))
    model.add(Dense(10, activation='softmax'))
    
    return model

In [14]:
def create_cnn_network(imgs_dim, compile_=True):
    """ Base network to be shared (eq. to feature extraction).
    """
    model = Sequential()

    model.add(_convolutional_layer(nb_filter=16, input_shape=imgs_dim))
    model.add(BatchNormalization(axis=1, mode=0))
    model.add(PReLU(init=W_INIT))
    model.add(_convolutional_layer(nb_filter=16))
    model.add(BatchNormalization(axis=1, mode=0))
    model.add(PReLU(init=W_INIT))
    model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='th'))

    model.add(_convolutional_layer(nb_filter=32))
    model.add(BatchNormalization(axis=1, mode=0))
    model.add(PReLU(init=W_INIT))
    model.add(_convolutional_layer(nb_filter=32))
    model.add(BatchNormalization(axis=1, mode=0))
    model.add(PReLU(init=W_INIT))
    model.add(_convolutional_layer(nb_filter=32))
    model.add(BatchNormalization(axis=1, mode=0))
    model.add(PReLU(init=W_INIT))
    model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='th'))



    
    
    model.add(Dropout(p=0.5))

    model.add(Flatten())
    model.add(_dense_layer(output_dim=PENULTIMATE_SIZE))
    model.add(BatchNormalization(mode=0))
    model.add(PReLU(init=W_INIT))


    
    return model

def _convolutional_layer(nb_filter, input_shape=None):
    if input_shape:
        return _first_convolutional_layer(nb_filter, input_shape)
    else:
        return _intermediate_convolutional_layer(nb_filter)


def _first_convolutional_layer(nb_filter, input_shape):
    return Conv2D(
        nb_filter=nb_filter, nb_row=3, nb_col=3, input_shape=input_shape,
        border_mode='same', init=W_INIT, W_regularizer=l2(l=L2_REG))


def _intermediate_convolutional_layer(nb_filter):
    return Conv2D(
        nb_filter=nb_filter, nb_row=3, nb_col=3, border_mode='same',
        init=W_INIT, W_regularizer=l2(l=L2_REG))


def _dense_layer(output_dim):
    return Dense(output_dim=output_dim, W_regularizer=l2(l=L2_REG), init=W_INIT)


In [29]:
from sklearn.model_selection import train_test_split
data = readin_small_data()
upselling_label = readin_label('upselling')
appetency_label = readin_label('appetency')

appetency_label = appetency_label.astype(int)

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=230))
#model.add(Dense(units=100, activation='relu'))
#model.add(Dense(units=100, activation='relu'))

model.add(Dense(units=1, activation='softmax'))

adam = Adam(lr=0.0001)
model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=adam,
        metrics=['accuracy'])



trainX, valX, trainY, valY = train_test_split(data, appetency_label, test_size=0.2, random_state=42)

model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data = (valX, valY))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10


ValueError: could not convert string to float: 'mj86'